In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 9.0 MB/s 
     |████████████████████████████████| 4.2 MB 50.5 MB/s 
     |████████████████████████████████| 212 kB 58.6 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 140 kB 61.9 MB/s 
     |████████████████████████████████| 84 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 41.8 MB/s 
     |████████████████████████████████| 127 kB 63.1 MB/s 
     |████████████████████████████████| 596 kB 53.6 MB/s 
     |████████████████████████████████| 6.6 MB 42.6 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 70.9 MB/s 
     |████████████████████████████████| 271 kB 68.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
model_checkpoint = "xlm-roberta-large" # "xlm-roberta-base" # "xlm-roberta-large" # "bert-base-multilingual-uncased" # "distilbert-base-uncased" 
batch_size = 4

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from datasets import load_dataset, load_metric

In [6]:
data_files = {"train": "/content/drive/My Drive/nlp/datasets/train2.json", "validation": "/content/drive/My Drive/nlp/datasets/test2.json"}
#datasets = load_dataset("json", data_files="/content/drive/My Drive/nlp/datasets/dev-v2.0.json")
datasets = load_dataset("json", data_files=data_files, field="data")

Using custom data configuration default-ce6d0ada85e19bdf


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ce6d0ada85e19bdf/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
datasets = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
datasets

DatasetDict({
    train: Dataset({
        features: ['answers', 'question', 'id', 'context', 'title'],
        num_rows: 86820
    })
    validation: Dataset({
        features: ['answers', 'question', 'id', 'context', 'title'],
        num_rows: 5928
    })
})

In [7]:
datasets["train"] = datasets["train"].select(list(range(0, 30000)))

In [9]:
datasets["validation"] = datasets["validation"].select(list(range(0, 1000)))

In [ ]:
datasets["test"] = datasets["validation"].select(list(range(1000, 5928)))

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

#!pip install transformers
#!pip install transformers[sentencepiece]
#from transformers import BertTokenizer, RobertaTokenizer, CamembertTokenizer, PreTrainedTokenizerFast
#import sentencepiece

#just SLO
#tokenizer = CamembertTokenizer.from_pretrained('drive/My Drive/ucenje_BERT/bert_pretrained_just_slo/sloberta/', return_offsets_mapping=True)

Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [10]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [11]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [12]:
pad_on_right = tokenizer.padding_side == "right"

In [13]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [14]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

#from transformers import BertForSequenceClassification, CamembertForQuestionAnswering, AdamW, BertConfig

#slo
#model = CamembertForQuestionAnswering.from_pretrained(
#    'drive/My Drive/ucenje_BERT/bert_pretrained_just_slo/sloberta/'
#)

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream t

In [16]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
)

In [17]:
from transformers import default_data_collator

data_collator = default_data_collator

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/Martin97Bozic/xlm-roberta-large-finetuned-squad into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 30542
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 7636


Epoch,Training Loss,Validation Loss


Saving model checkpoint to xlm-roberta-large-finetuned-squad/checkpoint-500
Configuration saved in xlm-roberta-large-finetuned-squad/checkpoint-500/config.json
Model weights saved in xlm-roberta-large-finetuned-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in xlm-roberta-large-finetuned-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in xlm-roberta-large-finetuned-squad/checkpoint-500/special_tokens_map.json
tokenizer config file saved in xlm-roberta-large-finetuned-squad/tokenizer_config.json
Special tokens file saved in xlm-roberta-large-finetuned-squad/special_tokens_map.json
Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.
Saving model checkpoint to xlm-roberta-large-finetuned-squad/checkpoint-1000
Configuration saved in xlm-roberta-large-finetuned-squad/checkpoint-1000/config.json
Model weights saved in xlm-roberta-large-finetuned-squad/checkpoint-1000/pytorch_model.bin
tokenizer conf

In [101]:
trainer.save_model("squad-multilingual-base-1-epoch")

Saving model checkpoint to squad-xml-base-whole-trained
Configuration saved in squad-xml-base-whole-trained/config.json
Model weights saved in squad-xml-base-whole-trained/pytorch_model.bin
tokenizer config file saved in squad-xml-base-whole-trained/tokenizer_config.json
Special tokens file saved in squad-xml-base-whole-trained/special_tokens_map.json
Saving model checkpoint to xlm-roberta-base-finetuned-squad
Configuration saved in xlm-roberta-base-finetuned-squad/config.json
Model weights saved in xlm-roberta-base-finetuned-squad/pytorch_model.bin
tokenizer config file saved in xlm-roberta-base-finetuned-squad/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-finetuned-squad/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.03G [00:00<?, ?B/s]

Upload file runs/May24_13-08-06_93ea1481c85f/events.out.tfevents.1653397695.93ea1481c85f.81.2:  42%|####1     …

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/Martin97Bozic/xlm-roberta-base-finetuned-squad
   726e582..4f81861  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/Martin97Bozic/xlm-roberta-base-finetuned-squad
   4f81861..3793488  main -> main



## Evaluation

In [41]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [42]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [43]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([123,  60,  16,  35,  34,  58, 182,  52,  97, 125, 150, 140,  28,  47,
          52,  67], device='cuda:0'),
 tensor([125,  62,  16,  36,  34,  62, 182,  55, 109, 129, 152, 140,  29,  49,
          53,  67], device='cuda:0'))

In [44]:
n_best_size = 20

In [45]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [46]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [47]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [48]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `XLMRobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `XLMRobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 6118
  Batch size = 16


In [49]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [50]:
max_answer_length = 30

In [51]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 14.145061, 'text': 'ci postopo'},
 {'score': 12.826483, 'text': 'potomci postopo'},
 {'score': 9.75216, 'text': 'postopo'},
 {'score': 8.494785, 'text': 'ci postopoma z združ'},
 {'score': 8.453041, 'text': 'po'},
 {'score': 7.381762, 'text': 'ci postopoma z z'},
 {'score': 7.176207, 'text': 'potomci postopoma z združ'},
 {'score': 7.0485697, 'text': 'ci'},
 {'score': 6.86769, 'text': 'ci postopoma z združitvijo'},
 {'score': 6.7107363, 'text': 'so njihovi potomci postopo'},
 {'score': 6.520057, 'text': 'ci posto'},
 {'score': 6.063184, 'text': 'potomci postopoma z z'},
 {'score': 5.7299914, 'text': 'potomci'},
 {'score': 5.549112, 'text': 'potomci postopoma z združitvijo'},
 {'score': 5.2641487, 'text': 'potom'},
 {'score': 5.201479, 'text': 'potomci posto'},
 {'score': 4.974487, 'text': 'ci postopoma'},
 {'score': 4.5429564,
  'text': 'ulskimi populacijami so njihovi potomci postopo'},
 {'score': 4.1018844, 'text': 'postopoma z združ'},
 {'score': 4.0500584, 'text': 'njiho

In [52]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [53]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    try:
                      if (
                          start_index >= len(offset_mapping)
                          or end_index >= len(offset_mapping)
                          or offset_mapping[start_index] is None
                          or offset_mapping[end_index] is None
                      ):
                          continue
                      # Don't consider answers with a length that is either < 0 or > max_answer_length.
                      if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                          continue

                      start_char = offset_mapping[start_index][0]
                      end_char = offset_mapping[end_index][1]
                      valid_answers.append(
                          {
                              "score": start_logits[start_index] + end_logits[end_index],
                              "text": context[start_char: end_char]
                          }
                      )
                    except:
                      print("error")
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)

        #if not squad_v2:
        #    predictions[example["id"]] = best_answer["text"]
        #else:
        #    answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        #    predictions[example["id"]] = answer

        #answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        #predictions[example["id"]] = answer
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [54]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 5928 example predictions split into 6118 features.


  0%|          | 0/5928 [00:00<?, ?it/s]

In [55]:
metric = load_metric("squad") #squad_v2

In [56]:
#id squad_v2 -> formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
#formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 36.45411605937922, 'f1': 53.3845112758235}

In [57]:
formatted_predictions[0:10]

[{'id': '56ddde6b9a695914005b9628', 'prediction_text': 'franciji'},
 {'id': '56ddde6b9a695914005b9629', 'prediction_text': '10 in 11 stoletju'},
 {'id': '56ddde6b9a695914005b962a', 'prediction_text': 'franciji'},
 {'id': '56ddde6b9a695914005b962b', 'prediction_text': 'rollo'},
 {'id': '56ddde6b9a695914005b962c', 'prediction_text': '10 stoletja'},
 {'id': '56dddf4066d3e219004dad5f', 'prediction_text': 'william osvajalec'},
 {'id': '56dddf4066d3e219004dad60', 'prediction_text': 'krona'},
 {'id': '56dddf4066d3e219004dad61', 'prediction_text': 'normanska'},
 {'id': '56dde0379a695914005b9636', 'prediction_text': 'normans normanz'},
 {'id': '56dde0379a695914005b9637', 'prediction_text': '9 stoletju'}]

In [58]:
import json
with open("predictions.json", "w") as predictions_file:
    json.dump(formatted_predictions, predictions_file)

In [59]:
references[0:10] 

[{'answers': {'answer_start': [133, 133, 133, 133],
   'text': ['franciji', 'franciji', 'franciji', 'franciji']},
  'id': '56ddde6b9a695914005b9628'},
 {'answers': {'answer_start': [82, 80, 82, 82],
   'text': ['10 in 11 stoletju',
    'v 10 in 11 stoletju',
    '10 in 11 stoletju',
    '10 in 11 stoletju']},
  'id': '56ddde6b9a695914005b9629'},
 {'answers': {'answer_start': [216, 216, 216, 216],
   'text': ['denmark ledu in norway',
    'denmark ledu in norway',
    'denmark ledu in norway',
    'denmark ledu in norway']},
  'id': '56ddde6b9a695914005b962a'},
 {'answers': {'answer_start': [267, 267, 267, 267],
   'text': ['rollo', 'rollo', 'rollo', 'rollo']},
  'id': '56ddde6b9a695914005b962b'},
 {'answers': {'answer_start': [544, 535, 82, 82],
   'text': ['10 stoletja', 'polovici 10 stoletja pojavila', '10', '10']},
  'id': '56ddde6b9a695914005b962c'},
 {'answers': {'answer_start': [956, 956, 956],
   'text': ['william osvajalec', 'william osvajalec', 'william osvajalec']},
  'id': '

In [60]:
import json
with open("references.json", "w") as references_file:
    json.dump(references, references_file)

In [100]:
def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [ ]:
compute_f1(formatted_predictions[4]['prediction_text'], references[4]['answers']['text'][0])

0.5

In [43]:
while True:
  continue

KeyboardInterrupt: ignored